In [90]:
from typing_extensions import Annotated
import autogen
from autogen import UserProxyAgent, AssistantAgent
import os
import re
import sys
import logging
import openai
from openai import OpenAI

In [110]:
# set your API key

os.environ['OPENAI_API_KEY']=""
api_key = os.getenv("OPENAI_API_KEY")
print(api_key)

# Configure logging at the start of your script
logging.basicConfig(level=logging.INFO)

# Workflow description




We want to optimize the colaboration between AI and Developer during Spring Deelopment followng Devonfw guidelines.

Given a Database Schema, we want a system to be concise, while accelerating the development process. Therefore we need the system to be as autonomous as possible, while being applicable to a large number of project. Thus the system needs a high amount of flexibility. As first experiments demonstrated, the state of LLMs is not advanced enough to autonomously make useful design decisions. Thus a high amount of flexibility must be met with human oversight. 
This new workflow, focusses on providing one "meta-workflow" with multiple "increment workflows" induced. The system will autonomously create files, write code and perform a review, but always asks for human feedback, or input for design decisions before performing a task. 

To make sure the communication between human and AI stays minimal, a global variable store is needed, that can be accessed by an agent in order to not ask questions repeatedly.

## 0. Meta Workflow
    1. Start the component workflow
    2. Repeat untill all components are implemented
## 1. Component workflow
The DB_schema.md file is structured into Business components. One business comnonent will be implemented by implementing all Entities inside the component.

    1. Determine the location of the business component
    2. Build the required blueprint structure following devon guidelines
    3. Start the Entity workflow
    4. Repeat 3. until all Entites are implemented
    5. Terminate

## 2. Entity Workflow
One entity consists of multiple parts and needs to  be organized into multiple files. 

    1. Determine location of Entity file
    2. Create Entity file
    3. Implement necessary library imports
    4. Deduce necessary imports from the relationships defined in db_schema.md
    5. Create DTO? Where?
    6. Does the DB_Schema suggest any additional common files like enums etc.?
    7. Review the implementation
       1. View list of generated files for the entity
       2. For every file, compare with requirements and devon guidelines
       3. Fill out Review.md file
       4. If problems found, pass on to coder
       5. Repeat Review workflow until no problems in review

## 3. Repository workflow
When the Entity files are implemented we ned a repository. The needed queries can by looking at the Db_schema (unuderstand the usage of the entity) and also the implementation.

    1. View requirements and implementation
    2. Suggest queries to human and ask for feedback
    3. implement repository

## 4. Test Entity implementation
The complete entity implementation needs to be tested. Thus we need information about all files that have been created for the entity and all functionalities.

    1. Gather information on
       1. Generated files
       2. Implemented relationships
    2. Generate testfile
    3. Implement testcases
    4. Execute Tests
    5. Review Results and Generate report
    6. If not all tests successfull, send report to coder and repeat until all tests succesfull



# Needed Agents



## Desicriptions of the agents

#### Meta Agent
#### Component Agent
#### Entity Coder Agent
- Global variables:
  - Filepath to dataaccess layer
- Tools:
  - list_dir
  - Read_file
  - Write file
  - Modify file?
  - StoreGlobal?
  - Ask_human

#### Random Coder Agent

#### DTO and Common Agent
- Global variables:
  - Dtos creation True, False, Ask
  - path to dataaccess layer/common folder
- Tools:
  - read_file
  - write_file
  
#### Review Agent
- Global variables:

- Tools:
  - read file

#### Code fix Agent
- Global variables:
  - Filepath to dataaccess layer

- Tools:
  - read_file
  - write_file / modify_file
  - 
  
#### Repository Agent
- Global variables:
  - Path to dataaccess layer

- Tools:
  - read_file
  - write_file
  - ask_human
  
#### Test Agent
  - Global variables:
    - what has been created

  - Tools:
    - Read file
    - Write file
    - run_test
    - 



# Prompting 



1. ReAct
2. Thought instruction

### Message templates

In [92]:
human_question = f"""
According to the review the agent suggests to reimplement the file.
Should I proceed with the reimplementation? If yes press enter, if no, type anything to stop the review loop
"""


system_message_entity_coder = """
You are an experienced SpringBoot developer adhering to devonfw guidelines. You write and insert code into an existing [Entity].java file, when given the required information.

Design guidelines you must follow:
Design a JPA/Hibernate Java entity with @Entity, and optionally @Table for naming (TableName Convention: always use singular.). Use @Column for mapping attributes to columns; ensure a no-argument constructor,
non-final classes/methods for Hibernate. Utilize standard Java types, custom types via AttributeConverter or @Embeddable. For Enums, use @Enumerated(EnumType.STRING);
large objects with @Lob, considering streaming for BLOBs. Handle dates with @Temporal. Define relationships with @ManyToOne, @OneToMany, @ManyToMany (lazy by default), 
@OneToOne (set to lazy), avoid bidirectional complexities, use Sets for collections, and Long with @GeneratedValue for primary keys. Implement embeddables with @Embeddable, 
choose @Inheritance strategy, typically SINGLE_TABLE. Entities should be simple, database-structure focused, avoiding business logic. 
Import Lombok for auto-generating getters/setters and annotate the Columns accordingly.
Use  import jakarta.persistence.* to efficiently import the library!

You always adhere to the following steps:
1. Think about the requirements and implementation guidelines given to you
2. Implement the Entity in the existing [Entity].java file directly using the respective function call. 
Implement the Entity EXACTLY as described in the requirements, do NOT change any names of attributes or relationships.	
3. When you´re done end the session with "TERMINATE".
When there are no further instructions write "TERMINATE"
"""

system_message_repository_coder = """


You are an experienced SpringBoot developer adhering to devonfw guidelines. You write and insert code into an existing [Entity]Repository.java file, when given the required information.
The repository should extend JpaRepository.
Make sure to import the entity and the JpaRepository from the correct package.

When you´re done end the session with "TERMINATE".
When there are no further instructions write "TERMINATE".


"""


system_message_coder = """
You are a SpringBoot developer following devon guidelines. You are tasked with implementing a Java file.
You are given the description for the files, the filenames, the path to the files and the package name. Use the respective function to implement those files accordingly.
Make sure to follow Java naming Conventions and the devonfw guidelines.

When you´re done end the session with "TERMINATE".
When there are no further instructions write "TERMINATE".

"""

system_message_redo_coder = """
You are a SpringBoot developer following devon guidelines. You are tasked with redoing the implementation of a file in order to make it suit the requirements and guidelines.

When you´re done end the session with "TERMINATE".
When there are no further instructions write "TERMINATE".
"""



## Implementation

In [93]:
spring_project_name = [x for x in os.listdir("..") if x != "Autogen"][0]

# append the name of the SpringBoot project to the index_path
spring_project_path = os.path.join("..", spring_project_name)

model = "gpt-4-1106-preview"

config_list = autogen.config_list_from_json(
    env_or_file="configurations.json",
    file_location="./",
    filter_dict={
        "model": [model]
    },
)




llm_config = {
    "config_list": config_list,
    "seed": 42,
    "timeout": 120,
    "temperature": 0.2
}


entity_coder= AssistantAgent(name ="entity_coder",
                                    system_message=system_message_entity_coder,
                                    human_input_mode="ALWAYS",
                                     llm_config={"config_list": config_list})
entity_proxy = UserProxyAgent(name="entity_proxy",
                            system_message="Terminate when agent says TERMINATE",
                            human_input_mode="NEVER",
                            is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
                             code_execution_config={"work_dir": "codingg", "use_docker": False}) # IMPORTANT: set to True to run code in docker, recommended

repo_coder= AssistantAgent(name ="repo_coder",
                                    system_message=system_message_repository_coder,
                                    human_input_mode="ALWAYS",
                                     llm_config={"config_list": config_list})

repo_proxy = UserProxyAgent(name="repo_proxy",
                            system_message="Terminate when agent says TERMINATE",
                            human_input_mode="NEVER",
                            is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
                             code_execution_config={"work_dir": "codingg", "use_docker": False}) # IMPORTANT: set to True to run code in docker, recommended

coder= AssistantAgent(name ="coder",
                        system_message=system_message_coder,
                        human_input_mode="ALWAYS",
                            llm_config={"config_list": config_list})

coder_proxy = UserProxyAgent(name="coder_proxy",
                            system_message="Terminate when agent says TERMINATE",
                            human_input_mode="NEVER",
                            is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
                             code_execution_config={"work_dir": "codingg", "use_docker": False}) # IMPORTANT: set to True to run code in docker, recommended

redo_coder= AssistantAgent(name ="redo_coder",
                                    system_message=system_message_redo_coder,
                                    human_input_mode="ALWAYS",
                                     llm_config={"config_list": config_list})
redo_proxy = UserProxyAgent(name="redo_proxy",
                            system_message="Terminate when agent says TERMINATE",
                            human_input_mode="NEVER",
                            is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
                             code_execution_config={"work_dir": "codingg", "use_docker": False}) # IMPORTANT: set to True to run code in docker, recommended

# Ways of launching agents


   

    1. Static agent: Follows strict procedure --> Static launch message, static system message, no global variables
   
    2. Dynamic Agent: Needs varying information depending on task. Information can be provided by:

       1. Dynamic System message
       2. Dynamic Chat initiation
       3. Global variables

    3. Hyper Dynamic Agent (concept): Same as dynamic, but also has dynamic access to functions
   
    

# Challenges



- Follow devon standards (how to induce them?)
- Follow user instructions
- Being able to roll back to a state if an error is detected

# Util Functions


1. List_dir
2. read_file
3. write_file
   

In [94]:
def list_directories() -> Annotated[str, "outputs file tree like tree command in cmd"]:

    path = spring_project_path
    directory_structure = ""

    for root, dirs, files in os.walk(path):
        # Filter out hidden directories
        dirs[:] = [d for d in dirs if not d.startswith('.')]
        files = [f for f in files if not f.startswith('.')]
        level = root.replace(path, '').count(os.sep)
        indent = ' ' * 4 * level
        directory_structure += '{}{}/\n'.format(indent, os.path.basename(root))
        subindent = ' ' * 4 * (level + 1)
        for f in files:
            directory_structure += '{}{}\n'.format(subindent, f)
    
    return directory_structure

def read_file(file_path: Annotated[str, "Relative path starting from the springboot directory to file location"]) -> str:
    """
    Extratcts all Columns, relationships and keys that are relevant for the entity out of the db_Schema.md file.
    Then it navigates to the correct java file and implements all the methods that are needed for the entity inside the existing file

    Parameters:
        entity_name (str): The name of the entity.
        location_of_file (str): The location of the file where the entity is implemented.
    """

    # helper to return the initial path provided by the llm
    initial_file_path=file_path

    # normalize the path changing forward slashes to backslashes
    file_path = file_path.replace("/", "\\")

    # check if first folder of the path is the last of the spring project folder before joining them
    if file_path.split("\\")[0] == spring_project_path.split("\\")[-1]:
        file_path = file_path.split("\\", 1)[1]

    # join the path of the spring project with the location of the file
    file_path = os.path.join(spring_project_path, file_path)

    # possible error handling for wrong path
    if not os.path.exists(file_path):
        tree = list_directories()
        return f"""The file does not exist at {initial_file_path}. This is our project structure: {tree} Please check the path and try again"""
    
    else:
        with open(file_path, "r") as f:
            content = f.read()

    return content

@coder_proxy.register_for_execution()
@redo_proxy.register_for_execution()
@entity_proxy.register_for_execution()
@repo_proxy.register_for_execution()
@coder.register_for_llm(description="writes code or text to a file at a specified path")
@redo_coder.register_for_llm(description="writes code or text to a file at a specified path")
@entity_coder.register_for_llm(description="writes code or text to a file at a specified path")
@repo_coder.register_for_llm(description="writes code or text to a file at a specified path")
def write_file(file_path: Annotated[str, "File path"], content: Annotated[str, "Content to be written to the file"]) -> Annotated[str, "Success message"]:
    """
    Writes the content to the file at the given file path.

    Parameters:
        file_path (str): The path to the file where the content should be written to.
        content (str): The content that should be written to the file.
    """
    initial_file_path=file_path

    # normalize the path changing forward slashes to backslashes
    file_path = file_path.replace("/", "\\")

    # check if first folder of the path is the last of the spring project folder before joining them
    if file_path.split("\\")[0] == spring_project_path.split("\\")[-1]:
        file_path = file_path.split("\\", 1)[1]

    # join the path of the spring project with the location of the file
    file_path = os.path.join(spring_project_path, file_path)

    # cut last part of the path to get the directory path
    directory_path = os.path.dirname(file_path)
    

    # possible error handling for wrong path
    if not os.path.exists(directory_path):
        tree = list_directories()
        return f"""The file does not exist at {initial_file_path}. This is our project structure: {tree} Please check the path and try again"""
    
    else:
        with open(file_path, "w") as f:
            f.write(content)

    return f"Content successfully written to {file_path}"


# Special purpose action functions

1. ask_human
2. get_component_name
3. get_entity_name
4. get_entity_requirements
5. create_component_structure
6. document_entity
7. mark_entity_done

In [95]:


def mark_component_done()-> Annotated[str, "Success message when component is marked as done"]:
    """
    Checks for the next unchecked component in the db_schema.md file and checks the next one as done.
    Returns a message indicating all components are checked if no unchecked component is found.
    """

    with open("./db_schema.md", 'r') as file:
        lines = file.readlines()

    # Flag to indicate whether a component has been marked as done
    component_marked = False

    # Iterate over each line in the file
    for i, line in enumerate(lines):
        # Check if a new unchecked component section starts
        if line.startswith("# [ ] Component:") and not component_marked:
            # Mark the component as done by changing the start of the line
            lines[i] = line.replace("[ ]", "[x]")
            component_marked = True
            break  # Exit the loop after marking a component

    # Write the updated lines back to the file
    with open("./db_schema.md", 'w') as file:
        file.writelines(lines)

    return "Component has been marked as done. TERMINATE"

def mark_next_entity_done()-> Annotated[str, "Success message when entity is marked as done"]:

    """
    Checks for the next unchecked entity in the db_schema.md file and checks the next one as done.
    Returns a message indicating all entities are checked if no unchecked entity is found.
    """

    with open("./db_schema.md", 'r') as file:
        lines = file.readlines()

    # Flag to indicate whether an entity has been marked as done
    entity_marked = False

    # Iterate over each line in the file
    for i, line in enumerate(lines):
        # Check if a new unchecked entity section starts
        if line.startswith("## [ ] Entity:") and not entity_marked:
            # Mark the entity as done by changing the start of the line
            lines[i] = line.replace("[ ]", "[x]")
            entity_marked = True
            break  # Exit the loop after marking an entity

    # Write the updated lines back to the file
    with open("./db_schema.md", 'w') as file:
        file.writelines(lines)

    return "Entity has been marked as done. TERMINATE"

def create_component_structure(component_base_path:Annotated[str, "Relative path from Spring project to where component should be created"])->str:
    """
    Creates a blueprint for a Spring component, handling deeper base paths.
    
    Parameters:
        component_base_path (str): The path inside the existing SpringBoot project, where the component should be created
    """
    
    # read the db schema file and to get then next component
    with open("./db_schema.md", 'r') as file:
        db_schema = file.read()


    # Find the first component name using a regular expression
    component_pattern = re.compile(r'# \[ \] Component: (\w+)')
    component_match = component_pattern.search(db_schema)

    if not component_match:
        return "All components implemented. TERMINATE"


    ### get list of all entities within the component, so stop the search when the next component starts
    
    # Extract the portion of the schema for the current component
    start_index = component_match.end()
    next_component_match = component_pattern.search(db_schema, start_index)
    if next_component_match:
        component_schema = db_schema[start_index:next_component_match.start()]
    else:
        component_schema = db_schema[start_index:]

    # Find all entity names within the component
    entity_pattern = re.compile(r'## \[ \] Entity: (\w+)')
    entities = entity_pattern.findall(component_schema)

    ### Create the component folder structure    
    component_base_path = component_base_path.replace("/", "\\")
    # check if first folder of the path is the last of the spring project folder before joining them
    if component_base_path.split("\\")[0] == spring_project_path.split("\\")[-1]:
        component_base_path = component_base_path.split("\\", 1)[1]

    # join the path of the spring project with the location of the file
    component_base_path = os.path.join(spring_project_path, component_base_path)

    # Check if the path exists
    if not os.path.exists(component_base_path):
        return	f"The path {component_base_path} does not exist. Please check the path and try again."
    
    component_name = component_match.group(1)

    # Normalize the base path by replacing backslashes with forward slashes and join with entity name
    component_path = os.path.join(component_base_path.replace('/', '\\'), component_name.lower())

    os.makedirs(component_path, exist_ok=True)

    # create the subfolders common, domain, logic, service
    subfolders = ['common', 'dataaccess', 'logic', 'service']
    for subfolder in subfolders:
        subfolder_path = os.path.join(component_path, subfolder)
        os.makedirs(subfolder_path, exist_ok=True)

    # create and repository folder inside domain and dto folder inside common
    dataaccess_path = os.path.join(component_path, 'dataaccess')
    repository_path = os.path.join(component_path, 'dataaccess', 'repository')
    dto_path = os.path.join(component_path, 'common', 'to')
    os.makedirs(dataaccess_path, exist_ok=True)
    os.makedirs(repository_path, exist_ok=True)
    os.makedirs(dto_path, exist_ok=True)

    # create the [Entity].java and [Entity]Repository.java files inside domain/model and domain/repository
    for entity in entities:

        # make sure the entity name starts with an uppercase letter
        entity = entity.capitalize()
        with open(os.path.join(dataaccess_path, f'{entity}.java'), 'w') as f:
            pass
        with open(os.path.join(repository_path, f'{entity}Repository.java'), 'w') as f:
            pass

        # create Dto.java file inside common/to
        with open(os.path.join(dto_path, f'{entity}Dto.java'), 'w') as f:
            pass
    # create component.java file inside logic
    with open(os.path.join(component_path, 'logic', f'{component_name.capitalize()}.java'), 'w') as f:
        pass

    return f"""The folders and files for {component_name} have been created. The [Entity].java files are located in {component_path}/dataaccess."""

In [96]:
def get_entity_name()-> Annotated[str, "Name of the next entity to be implemented"]:

    with open("./db_schema.md", 'r') as file:
        db_schema = file.read()

    # Find the first component name using a regular expression
    component_pattern = re.compile(r'# \[ \] Component: (\w+)')
    component_match = component_pattern.search(db_schema)
    
    # Extract the portion of the schema for the current component
    start_index = component_match.end()
    next_component_match = component_pattern.search(db_schema, start_index)
    if next_component_match:
        component_schema = db_schema[start_index:next_component_match.start()]
    else:
        component_schema = db_schema[start_index:]

    # Find next unchecked within the component
    entity_pattern = re.compile(r'## \[ \] Entity: (\w+)')

    # only get next unchecked Entity
    next_entity = entity_pattern.search(component_schema).group(1)
    
    if not next_entity:
        # mark the component done by changing the first component_match to [x]
        db_schema = db_schema.replace(component_match.group(0), component_match.group(0).replace("[ ]", "[x]"), 1)
        return "All entities are implemented, component has been marked as done. TERMINATE"

    # Return the first match if any, else None
    return next_entity 

def get_entity_requirements() -> Annotated[str, "Requirements for current the entity"]:
    # Open and read the markdown file
    with open("./db_schema.md", 'r') as file:
        db_schema = file.read()

    # Pattern to identify the start of the first unchecked component
    component_pattern = re.compile(r'# \[ \] Component: \w+')
    component_start_match = component_pattern.search(db_schema)
    
    if not component_start_match:
        return "All components are implemented. TERMINATE"
    
    # Narrow down the search to start from the first unchecked component
    db_schema_from_first_component = db_schema[component_start_match.start():]

    # Pattern to find the first unchecked entity after the first unchecked component
    # Ensuring it only captures up to the start of the next entity or the end of the component
    entity_pattern = re.compile(r'## \[ \] Entity: (\w+)(.*?)(?=## \[ \]|# \[ \]|$)', re.DOTALL)
    entity_match = entity_pattern.search(db_schema_from_first_component)

    if entity_match:
        entity_name = entity_match.group(1)
        details = entity_match.group(2).strip()
        full_text = f"Entity name: {entity_name}\nDetails: {details}"
        return full_text
    else:
        return "No unchecked entities found. TERMINATE"

def get_component_name() -> Annotated[str, "Name of the next component to be implemented"]:
    """
    Extracts the name of the next component that needs to be implemented from the db_schema.md file.

    Returns:
        str: The name of the next component that needs to be implemented.
    """

    with open("./db_schema.md", 'r') as file:
        db_schema = file.read()

    # Find the first component name using a regular expression
    component_pattern = re.compile(r'# \[ \] Component: (\w+)')
    component_match = component_pattern.search(db_schema)
    
    # Return the first match if any, else None
    return component_match.group(1).lower() if component_match else "All components are implemented. TERMINATE"



### Message Templates

In [97]:
def get_special_files_message():

    entity_req = get_entity_requirements()
    entity_name = get_entity_name()

    system_message = f"""You are an experienced SpringBoot architect adhering to devonfw standards.
      Based on requirements for an entity, you determine if any files need to be created inside the common folder (e.g. enums).
    """
    message = f"""These are the requirements for the entity {entity_name}: {entity_req}
            Do the Requirements suggest that we need to create any enum files in the common folder for this Entity?
             If yes, list them inside << >> like this: <<filenames>> and provide a short description of the files.
             Naming convention: The filenames should ALWAYS have the Entity name: {entity_name} as a prefix.
             Make sure the filenames are separated by a comma and a space and end with .java.

             If no additional files are needed, reply "No additional files needed for this entity."

             If files are needed, structure your reply like this:
             Structure your reply like this: 
             <<filenames>>
             ####file: description of file.####
             ####file: description of file.####
             ...
             """
    
    return system_message, message

def get_review_sys_message():

    review_structure = open("review_blueprint.md", "r").read()
    system_message = f"""
    You are a code reviewer that makes sure requirements and conventions are implemented as desired.
    You ALWAYS reply with a review according to the structure: {review_structure}
    """
    return system_message

def get_create_enum_message(completion, common_path, package_name_new):
    
    message = f"""Implement this description of files:
                                  {completion} exactly as described, in the {common_path} folder using the respective function. 
                                  The package names are {package_name_new}, make sure to include them in your files. 
                                  Also make sure to use public enum when createing an enum.
                                   After you implemented all files, reply "TERMINATE". """
    
    return message

def get_review_message(file_name, requirements, implementation):

    entity_name = get_entity_name()

    message = f"""
    We are currently working on the entity: {entity_name}.
    Therefore we created the file {file_name}.
    Does this implementation <begin implementation> {implementation} <end implementation>
    meet ALL of the requirements: {requirements}?
    """

    return message

def get_redo_message(file_name, entity_req, file_path, review_report):

    entity_name = get_entity_name()
    implementation = read_file(file_path)

    entity_redo = f"""
    We are currently working on the entity: {entity_name}.
    Therefore we created the file {file_name}.
    These were the requirements for the file: {entity_req}. 
    This is the current implementation: {implementation}.
    The implementation is flawed, as this review report suggests: {review_report}. 
    Please redo the implementation according to the requirements and the review report.
    Use the according function and insert your codeat {file_path}
    """

    return entity_redo

def get_entity_message(full_entity_path, entity_req, package_name, enums):

    message = f"""The path to the entity file is {full_entity_path} and the requirements are {entity_req}.
        The package name will be {package_name}.
        Implement the entity according to the requirements and the package name. Make sure to import all packages that you refer to in your code.
        {enums} 
        """
    return message

def get_dto_message(entity_impl, package_name, file_name, path, enums_message):
    entity_name = get_entity_name()
    message = f"""Implement the DTO for the entity according to following implementation of the entity: {entity_impl} file. The package name will be {package_name} and the filename will be {file_name}.
                                Create the file at {path}.{enums_message}. Also make sure to import lombok and annotate the Dto with @Getter and @Setter.
                                Do not forget to import the {entity_name}.java file.
                               """
    
    return message

def get_repo_query_message(entity_impl):
    entity_name = get_entity_name()
    entity_req = get_entity_requirements()

    message = f"""Based on the requirements for the entity {entity_name} and the current implementation, suggest queries that can be used to retrieve data from the database.
    Keep in mind that JPA methods like findAll exist, so do not make unnecessary suggestions.
      The requirements for the entity are:
      {entity_req} The current implementation of the entity is: {entity_impl}"""

    sys_message = "You are a springboot architect that suggests repository queries."
    return sys_message, message


# Workflow functions

1. Launch_component_workflow
2. Launch_entity_workflow
3. Launch_Repository workflow
4. Launch_testing_workflow

In [98]:

def ask_human(question: Annotated[str, "Question to be asked to the human agent"]) -> Annotated[str, "Answer"]:
  answer = input(f"""Please answer the question: {question}\n""")
  if answer == "TERMINATE":
      sys.exit()


  return answer

def get_usage(res):
    prompt_tokens_used = res.usage.prompt_tokens
    completion_tokens_used = res.usage.completion_tokens
    cost = prompt_tokens_used*(10.00 / 1e6) + completion_tokens_used*(30.00 / 1e6)
    print("prompt tokens used: ", prompt_tokens_used)
    print("completion tokens used: ", completion_tokens_used)
    print(f"""Cost: {cost} USD for GPT-4""")

    return

def get_openai_reply(system_message, message):

    client = OpenAI()
    if type(message)== str:

        messages = [
            {"role": "system", "content": system_message},
            {"role": "user", "content": message}
        ]

    else:
        messages = message

    res = client.chat.completions.create(
        model=model,
        messages= messages, 
        temperature=0.2
    )
    completion = res.choices[0].message.content

    messages.append({"role": "assistant", "content": completion})

    get_usage(res)

    return completion, messages

def analyze_human_input(human_input, messages):

    task = f""" Your boss reviewed your answer and wrote some feedback.
      Please provide a new answer, following exactly the previous structure, but with the new input from your boss in mind. What he says is LAW. This is the feedback: {human_input}"""

    messages.append({"role": "user", "content": task})

    completion, new_messages = get_openai_reply("x", messages)
    return completion, new_messages

def get_special_files_sugg():
    system_message, message = get_special_files_message()
    completion, messages = get_openai_reply(system_message, message)
    return completion, messages



In [99]:


def get_agent_usage(assistant_name, proxy_name):

    print("Usage  ",proxy_name.name, ": ", coder_proxy.print_usage_summary())
    print("Usage  ",assistant_name.name, ": ", assistant_name.print_usage_summary())
    return 



def get_path():
    tree = list_directories()
    client = OpenAI()
    res = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are an AI assistant. You are helping a user to navigate to a specific directory in a SpringBoot project.  "},
            {"role": "user", "content": f"""This is the structure of my current directory: {tree}
                Based on this structure, infer the path to the Spring Boot project directory, where a business component would be created inside the Spring Boot project It is usually the same directory where the Application file is located. Enclose the path in special markers like so: <<file_path>>. The path to the directory is:"""},
            
        ]
    )
    completion = res.choices[0].message.content
    
    # parse the path from the completion, it is wrapped in << >>
    completion = completion.split("<<")[1].split(">>")[0]
    # write path to file for global storage
    with open('spring_project_path.txt', 'w') as f:
        f.write(completion)

    

    return f""" The path {completion} has been written to the file spring_project_path.txt"""

def get_package_name(location_of_file):
    # if com in path, remove everything before it
    if f"""\\com\\""" in location_of_file:
        package_name = location_of_file[location_of_file.find(f"""\\com\\"""):]
        # remove last part of path
        package_name = package_name[:location_of_file.rfind(f"""\\""")]

        package_name = package_name.replace(f"""\\""", ".")
        if package_name.startswith("."):
            package_name = package_name[1:] 

    else: 
        package_name = ask_human("Please provide the package name for the entity.")

    return package_name

def suggest_queries(entity_path):

    # get implementation of entity
    entity_impl = read_file(entity_path)

    system_message, message = get_repo_query_message(entity_impl)

    completion, messages = get_openai_reply(system_message, message)
    print("+++++suggested queries+++++")
    print(completion)
    
    feedback = ask_human("Please review the suggestions and give feedback. If everything looks fine, press enter.")
    
    # Loop for refinements based on feedback
    while feedback:

        completion = analyze_human_input(feedback, messages)[0]
        print("+++++suggested new queries+++++")
        print(completion)
        feedback = ask_human("Please review the suggestions and give feedback. If everything looks fine, press enter.")

    return completion
          


def document_entity():
    pass

def append_file_to_entity_files(name_of_file):
    # get current list of files
    with open("./entity_files.py", 'r') as file:
        files = file.read()
    
    if files:
        list(files).append(name_of_file)
    else:
        files = [name_of_file]

    with open("./entity_files.py", 'w') as file:
        file.write(str(files))

    return


In [100]:
def get_req_ent(package_name, enums):
    # entity requirements general (lombok, jakarta.persistence)
    # special requirements: uses the following enums:...
    entity_req = get_entity_requirements()
    
    entity_req_message = f"""
    Devon guidelines: 
    A JPA/Hibernate Java entity shall be designed, with @Entity, and optionally @Table for naming. Use @Column for mapping attributes to columns; ensure a no-argument constructor,
    non-final classes/methods for Hibernate. Table names should always be singuar. Utilize standard Java types, custom types via AttributeConverter or @Embeddable. For Enums, use @Enumerated(EnumType.STRING);
    large objects with @Lob, considering streaming for BLOBs. Handle dates with @Temporal. Define relationships with @ManyToOne, @OneToMany, @ManyToMany (lazy by default), 
    @OneToOne (set to lazy), avoid bidirectional complexities, use Sets for collections, and Long with @GeneratedValue for primary keys. Implement embeddables with @Embeddable, 
    choose @Inheritance strategy, typically SINGLE_TABLE. Entities should be simple, database-structure focused, avoiding business logic. Use Lombok for auto-generating getters/setters,
    with @Getter and @Setter annotation.

    The requirements for the entity are: {entity_req}
    And the file should implement the package {package_name} and use the enums, which were already created: {enums}
    """

    return entity_req_message

def get_req_enum(num_enum, package_name):
    # requirements for enums: stored in a file
    enum_reqs = open("enum_files.txt", "r").read()

    # Adjusting the split logic to properly separate entries
    sections = enum_reqs.split('<<')[1:]  # Skip the first empty entry if any

    # Validate the requested enum number
    if num_enum < 0 or num_enum >= len(sections):
        return "Invalid number for file and description in get_req_enum"
    
    # Extract the filename and description with an adjusted regex pattern
    pattern = r'(.*?)>>\s*####(.*?): (.*?)####'
    match = re.search(pattern, sections[num_enum], re.DOTALL)
    if match:
        filename = match.group(1).strip()
        description = match.group(3).strip()
        enum_req = filename + ": " + description
        
        enum_req_message = f"""
        The enum will be used by the entity and will implement the following requirements: {enum_req}
        Inside the package {package_name}
        """
        return enum_req_message
    else:
        return "Error in parsing file and description"
 
def get_req_dto(entity_impl, package_name, enums_message):
    entity_name = get_entity_name()
    f"""The DTO for the entity {entity_name} should be implemented according to following implementation of the entity: {entity_impl} file. The package name should be {package_name}.
                                {enums_message}. Also make sure to import lombok and annotate the Dto with @Getter and @Setter.
                                Also the implementation should strictly adhere to Java naming conventions and Devon guidelines.
                               """

def get_req_repo(package_name, suggestions):
    entity_name = get_entity_name()[1]
    repo_req = f"""
    The requirements for the repository are: {suggestions}
    The repository should implement the package {package_name} for the Entity {entity_name}
    The repository should be created with the name {entity_name}Repository extending JpaRepository.
    """
    return repo_req




In [107]:
def special_request_workflow():
    pass

def launch_review_workflow(file_path, requirements, file_name):
  
    implementation = read_file(file_path)

    system_message = get_review_sys_message()
    review_message = get_review_message(file_name, requirements, implementation)
    review, messages = get_openai_reply(system_message, review_message)   
     
    print("+++++++++++++++ FIRST REVIEW +++++++++++++++ \n")
    logging.info(review)
    # launch loop for rework
    while "Needs rework: Yes" in review:
        
        print("++++++++++++++entering rework loop+++++++++++++")
        # ask for feedback
        feedback = ask_human(human_question)

        if feedback:
            review, messages = analyze_human_input(feedback, messages)

            print("++++++++++++++REWORKED REVIEW+++++++++++++\n ")
            logging.info(review)
            if "Needs rework: No" in review:
                break

        redo_proxy.initiate_chat(redo_coder, message = get_redo_message(file_name, requirements, file_path, review))
        implementation = read_file(file_path)
        
        review_message = get_review_message(file_name, requirements, implementation)
        review, messages = get_openai_reply(system_message, review_message)  
        logging.info(review)
                
    
    return "++++++++++++++++Review of special files complete.+++++++++++++++++"

def launch_testing_workflow():
    pass

def launch_repository_workflow(entity_path, package_name):

    # get entity name from the path
    entity_name = get_entity_name()

    # infer useful queries from the entity requirements and implementation
    suggestions = suggest_queries(entity_path)
    # remove entity from path to get the path to the repository
    repo_path = entity_path[:entity_path.rfind("\\")] + "\\repository"
    package_name = package_name + "repository"
    file_name = entity_name + "Repository.java"
    # launch the repository coder
    repo_proxy.initiate_chat(repo_coder, message=f"""The current implementation of the entity is:
                              {read_file(entity_path)}. Create the repository file for the entity {entity_name} in the package {package_name} At the location {repo_path} under the name {file_name}.
                              Implement the suggested queries: {suggestions}. And don´t forget to import the entity from {entity_path}.""")

    # get usage summary of the agents
    get_agent_usage(repo_coder, repo_proxy)

    # append the file to the list of entity files
    #append_file_to_entity_files(file_name)

    # initiate review workflow  
    repo_req = get_req_repo(package_name, suggestions)

    print("starting Repository review workflow ")
    launch_review_workflow(repo_path + "\\" + file_name, repo_req, file_name)


    return entity_name, "Repository has been created."

def launch_dto_workflow(entity_path, common_path, package_name, enums):

    entity_name = get_entity_name()
    if enums == "No additional files are needed":
        enums = ""
    else:
        enums_message = f""" The files {enums} have been created in the common folder, import them in your implementation from {common_path}"""
        

    # read the entity file
    entity_impl = read_file(entity_path)

    # create package name for the dto by removing the last part of the package name and adding .common.to
    package_name = package_name[:package_name.rfind(".")] + ".common.to"
    path = common_path + "\\to"

    file_name = entity_name + "Dto.java"

    message = get_dto_message(entity_impl, package_name,file_name, path, enums_message)
    # launch the dto coder
    coder_proxy.initiate_chat(coder,message = message)

    get_agent_usage(coder, coder_proxy)

    # review workflow
    entity_req = get_req_dto(entity_impl, package_name, enums_message)

    print("+++++++++++++++++++starting DTO review workflow++++++++++++++++++++")
    launch_review_workflow(path + "\\" + file_name, entity_req, file_name)


    # append the file to the list of entity files
    append_file_to_entity_files(file_name)
    return "DTO has been created."

def launch_entity_impl_worlflow(full_entity_path, entity_req, package_name, enums, entity_name):

    message = get_entity_message(full_entity_path, entity_req, package_name, enums) 

    entity_proxy.initiate_chat(entity_coder, message= message)
    
    # print usage summary of the agents
    get_agent_usage(entity_coder, entity_proxy)

    append_file_to_entity_files(entity_name+".java")

    # initiate review workflow
    entity_req = get_req_ent(package_name, enums)

    print("+++++++++++++++++++starting Entity review workflow for: ", entity_name, "++++++++++++++++++++")
    launch_review_workflow(full_entity_path, entity_req, entity_name)


    return f"""+++++++++++++++Entity file for {entity_name} has been created. Proceed with DTO workflow.+++++++++++"""

def create_enums(completion, list_of_files, common_path):

        package_name = get_package_name(common_path)
        # create the files in the common folder
        coder_proxy.initiate_chat(coder, message = get_create_enum_message(completion, common_path, package_name))
        # print the usage summary of the agents
        get_agent_usage(coder, coder_proxy)

        # store the completion in a file for later reference
        with open('enum_files.txt', 'w') as f:
            f.write(completion+ "The package name for all files is: " + package_name + " . The path to the common folder is: " + common_path)

        # write the list of files to a file for later reference
        with open('entity_files.py', 'w') as f:
            f.write("files = "+ str(list_of_files))


def special_files(common_path):
    
    package_name = get_package_name(common_path)
    completion, messages = get_special_files_sugg()
    logging.info(f"Suggestion: {completion}")
    # ask human if the suggestion is correct
    human_input = ask_human(f"""The AI suggests: {completion} Is this correct? If yes hit enter, if no, please provide the correct description using "<<filenames>>" for the filenames.""")

    if human_input:
        completion = analyze_human_input(human_input, messages)[0]

    # if list of files is not empty
    if "<<" in completion:
        # get the list of files from the completion
        list_of_files = completion.split("<<")[1].split(">>")[0].split(", ")   

        print("The following list of files will be created:", list_of_files)

        create_enums(completion, list_of_files, common_path)

        for i in range(0, len(list_of_files)):
            req = get_req_enum(i, package_name)
            path = common_path + "\\" + list_of_files[i]
  
            print("starting enum review workflow for: ", list_of_files[i])
            launch_review_workflow(path, req, list_of_files[i])

        if len(list_of_files) > 1:
            enums = f"""The following files have been created for the entity. Import them in your implementation from {common_path} + filename(s): {list_of_files}"""
        else:
            enums = f"""The following file has been created for the entity. Import it in your implementation from {common_path} + filename: {list_of_files}"""


    else: 
        # create empty entity_files.py file, overwriting the old one if exists
        with open('entity_files.py', 'w') as f:
            f.write("files = []")

        enums = ""

    return enums



In [102]:
def launch_entity_workflow(component_name):
    
    # get path to spring project
    spring_path = open('spring_project_path.txt', 'r').read()    

    # get the path to the common folder
    common_path = os.path.join(spring_path, component_name, "common")

    # replace forward with back slashes
    common_path = common_path.replace("/", "\\")

    while True:
        entity_name = get_entity_name()
        if entity_name == "All entities are implemented, component has been marked as done. TERMINATE":
            return entity_name

        else:

            entity_path = os.path.join(spring_path, component_name, 'dataaccess', f'{entity_name}.java').replace("/", "\\")

            ##### check if any special requests exist (flexibility to be implemented######
            package_name = get_package_name(entity_path)
            # get the entity requirements
            entity_req = get_entity_requirements()
            
            print("++++++++++++++++starting special file workflow+++++++++++++++")
            enums = special_files(common_path) 
        
            print("+++++++++++++++++starting entity implementation workflow++++++++++++++")
            launch_entity_impl_worlflow(entity_path, entity_req, package_name, enums, entity_name)

            print("+++++++++++++++++starting DTO workflow++++++++++++++++++++")
            launch_dto_workflow(entity_path, common_path, package_name, enums)

            print("++++++++++++++++++starting repository workflow++++++++++++++++++")
            launch_repository_workflow(entity_path, package_name)

            print("++++++++++++++++++ENTITY DONE++++++++++++++++++++")
            mark_next_entity_done()

            print("++++++++++++++++++STARTING", entity_name, "++++++++++++++++++")

def launch_component_workflow():
    try:

        # get the path to the spring project and store it once
        get_path()

        # loop through the components
        while True:

            component_name = get_component_name()
            
            if component_name == "All components are implemented. TERMINATE":
                return component_name
            
            else:
                # ask clarifying questions
                clarifying_question ="I will create the component structure according to the devon guidelines. If you have a special request, let me know, otherwise press enter to proceed."
                answer = ask_human(clarifying_question)
                if answer:
                    special_request_workflow()

                else:
                    result = create_component_structure(open('spring_project_path.txt', 'r').read())
                    while "does not exist" in result:
                        print(result)  # Inform the user of the issue.
                        get_path()  # Obtain a new path.
                        result = create_component_structure(open('spring_project_path.txt', 'r').read())  # Retry with the new path.
                    
                    # If this point is reached, the component was created successfully.
                    print(result)



                launch_entity_workflow(component_name = component_name)

    except SystemExit:
        print("Program terminated by user.")

        

    

In [109]:
launch_component_workflow()     

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


The folders and files for CourseManagement have been created. The [Entity].java files are located in ..\Gabis-MVP SpringBoot\src\main\java\com\gabismvp\coursemanagement/dataaccess.
++++++++++++++++starting special file workflow+++++++++++++++


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Suggestion: <<CourseLevel.java>>
####file: This file would define an enum for the Course Level attribute, which includes the values Beginner, Intermediate, and Advanced.####


prompt tokens used:  409
completion tokens used:  32
Cost: 0.005050000000000001 USD for GPT-4
The following list of files will be created: ['CourseLevel.java']
coder_proxy (to coder):

Implement this description of files:
                                  <<CourseLevel.java>>
####file: This file would define an enum for the Course Level attribute, which includes the values Beginner, Intermediate, and Advanced.#### exactly as described, in the src\main\java\com\gabismvp\coursemanagement\common folder using the respective function. 
                                  The package names are com.gabismvp.coursemanagement.common, make sure to include them in your files. 
                                  Also make sure to use public enum when createing an enum.
                                   After you implemented all files, reply "TERMINATE". 

--------------------------------------------------------------------------------

>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


coder (to coder_proxy):

***** Suggested tool Call (call_PPQEnS9pC49etGKMbzCqDPGR): write_file *****
Arguments: 
{
  "file_path": "src\\main\\java\\com\\gabismvp\\coursemanagement\\common\\CourseLevel.java",
  "content": "package com.gabismvp.coursemanagement.common;\n\npublic enum CourseLevel {\n    BEGINNER,\n    INTERMEDIATE,\n    ADVANCED\n}"
}
***************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION write_file...
coder_proxy (to coder):

coder_proxy (to coder):

***** Response from calling tool "call_PPQEnS9pC49etGKMbzCqDPGR" *****
Content successfully written to ..\Gabis-MVP SpringBoot\src\main\java\com\gabismvp\coursemanagement\common\CourseLevel.java
**********************************************************************

--------------------------------------------------------------------------------
coder (to coder_proxy):

TERMINATE

--------------

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:# Code Review for CourseLevel.java
Needs rework: Yes

## Functionality

### Syntax or logic errors
None

### Adherence to requirements

All requirements met: No

Description of problems:
- The enum values are in uppercase, whereas the requirements specify that the values should be "Beginner", "Intermediate", and "Advanced" with only the first letter capitalized.

Suggested Improvements:
- Change the enum values to have only the first letter capitalized as per the requirements. The enum should look like this:

```java
public enum CourseLevel {
    Beginner,
    Intermediate,
    Advanced
}
```

### Naming Conventions
Java naming conventions met according to devon guidelines: Yes

Java enums typically have constants in all uppercase letters, which is a common practice. However, since the specific requirement is to have the values with only the first letter capitalized, the naming conventi

prompt tokens used:  258
completion tokens used:  188
Cost: 0.00822 USD for GPT-4
+++++++++++++++ FIRST REVIEW +++++++++++++++ 

++++++++++++++entering rework loop+++++++++++++
Program terminated by user.


# Agents